In [134]:
import os
import requests
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import warnings
warnings.filterwarnings(action='ignore')
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [135]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 90% !important; }</style>"))

In [136]:
def get_wiki_data(query):

    # Define SPARQL endpoint
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    headers = {
        "User-Agent": "MySPARQLQueryBot/1.0 (your_email@example.com)"  # Use your own email for compliance
    }
    
    # Create a SPARQLWrapper instance
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addCustomHttpHeader("User-Agent", "MySPARQLQueryBot/1.0 (your_email@example.com)")
    
    ### Get Responce from WIKI query service
    response = sparql.query().response.read().decode("utf-8")  # Decode the response as UTF-8
    
    ### Convert to JSON
    data = json.loads(response)
    results = data["results"]["bindings"]
    return results

In [146]:
query_temp = """

SELECT DISTINCT ?person ?personLabel ?birthPlaceLabel ?birthDate ?coordinates ?deathPlaceLabel 
        ?coordinates_death ?deathDate ?wikipedia ?sexLabel ?ukraineRelatedEntityLabel 
WHERE {{

  ?person wdt:P31 wd:Q5.  # instance of human

  VALUES ?person {{ {items} }}

  # Wikipedia article in Ukrainian
  ?wikipedia schema:about ?person;
             schema:isPartOf <https://uk.wikipedia.org/>.

  # Birth info
  ?person wdt:P19 ?birthPlace.
  ?birthPlace wdt:P625 ?coordinates.
  ?person wdt:P569 ?birthDate.

  OPTIONAL {{ ?person wdt:P27 ?ukraineRelatedEntity. }}
  
  # Optional death info
  OPTIONAL {{ ?person wdt:P20 ?deathPlace. 
                     ?deathPlace wdt:P625 ?coordinates_death. }}
  OPTIONAL {{ ?person wdt:P570 ?deathDate. }}

  OPTIONAL {{ ?person wdt:P21 ?sex. }}

  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "uk". }}
}}
LIMIT 30000
"""

query = query_temp.format(items=items)

In [181]:
### Read data extracted from https://petscan.wmcloud.org/
### Search query: 
### "українське походження" OR "українського походження" OR "українських емігрантів" OR "українського емігрантського" OR "етнічний українець" OR "етнічна українка" OR "українка за походженням" OR "українець за походженням" OR "в родині українців"

items_df = pd.read_csv('download_from_petscan.csv')
items_df.Wikidata = "wd:" + items_df.Wikidata.astype(str)
unique_items = items_df.Wikidata.unique()
items = " ".join(uniqitems.tolist())
print(len(unique_items))

3997


In [189]:
import requests
import pandas as pd
import json

### Define queary before usage using search input and extract psid from: https://petscan.wmcloud.org/
### Search query: 
### "українське походження" OR "українського походження" OR "українських емігрантів" OR "українського емігрантського" OR "етнічний українець" OR "етнічна українка" OR "українка за походженням" OR "українець за походженням" OR "в родині українців"

psid = 34986493
url = f"https://petscan.wmflabs.org/?psid={psid}&format=json"

response = requests.get(url)
data = response.json()

# Pages are usually here:
try:
    pages = data['*'][0]['a']["*"]
except (KeyError, IndexError, TypeError):
    pages = []

# Build a DataFrame
results = []
for page in pages:
    if isinstance(page, dict):
        results.append({
            'Title': page.get('title'),
            'Pageid': page.get('id'),
            'Wikidata': page.get('q')
        })

items_df = pd.DataFrame(results)
items_df.Wikidata = "wd:" + items_df.Wikidata.astype(str)
unique_items = items_df.Wikidata.unique()
print('Dataframe size:', items_df.shape)

items_df.head()

Dataframe size: (4000, 3)


,Title,Pageid,Wikidata
0,1908,622,wd:Q2056
1,1904,625,wd:Q2046
2,1979,640,wd:Q2484
3,1900,671,wd:Q2034
4,1919,674,wd:Q2157


In [190]:
s = 100
result_all = pd.DataFrame()
for x in range(0, int(len(unique_items)/100)+2):
    temp = unique_items[x*100: (x+1)*100]
    temp_str = ' '.join(temp)
    query = query_temp.format(items=temp_str)
    results_json = get_wiki_data(query=query)

    # Convert results to a pandas DataFrame
    df = pd.DataFrame([
        {
            "Coordinates": item.get("coordinates", {}).get("value", ""),
            "PersonName": item.get("personLabel", {}).get("value", ""),
            "BirthPlace": item.get("birthPlaceLabel", {}).get("value", ""),
            "BirthDate": item.get("birthDate", {}).get("value", ""),
            "DeathDate": item.get("deathDate", {}).get("value", ""),
            "DeathPlace": item.get("deathPlaceLabel", {}).get("value", ""),
            "Coordinates_death": item.get("coordinates_death", {}).get("value", ""),
            "Sex": item.get("sexLabel", {}).get("value", ""),
            "WikipediaURL": item.get("wikipedia", {}).get("value", ""),
            "Affiliation": item.get("ukraineRelatedEntityLabel", {}).get("value", "Україна"),
        }
        for item in results_json
    ])


    result_all = result_all._append(df)
    print('Count:', len(results_json))
print('Total result:', result_all.shape)

Count: 4
Count: 46
Count: 93
Count: 69
Count: 107
Count: 31
Count: 48
Count: 8
Count: 41
Count: 51
Count: 88
Count: 66
Count: 71
Count: 55
Count: 70
Count: 58
Count: 83
Count: 74
Count: 90
Count: 73
Count: 73
Count: 63
Count: 68
Count: 65
Count: 52
Count: 72
Count: 86
Count: 74
Count: 114
Count: 53
Count: 78
Count: 71
Count: 71
Count: 95
Count: 62
Count: 86
Count: 72
Count: 80
Count: 70
Count: 82
Count: 0
Count: 0
Total result: (2713, 10)


In [191]:
result_all = result_all.drop_duplicates(['PersonName','WikipediaURL'])
result_all.to_excel('petscan_data.xlsx', index=False)